In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import random
from torch_geometric.data import Data

In [3]:
#select unkown genes which are not positive samples either negative samples
df1 = pd.read_table('../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt', index_col=0, header =0, sep="\t")
gene_index = list(df1.index.values)
df2 = pd.read_table('../data/796true.txt', header=None, index_col=None, names=['name'], dtype='str')
df3 = pd.read_table('../data/2187false.txt', header=None, index_col=None, names=['name'], dtype='str')

driver_index = []
normal_index = []
unkown_index = []
unkown_gene_name = []
for index,gene in enumerate(gene_index):
    flag = 0
    for driver in list(df2['name'].values):
        if gene==driver:
            driver_index.append(index)
            flag = 1
    for normal in list(df3['name'].values):
        if gene==normal:
            normal_index.append(index)
            flag = 1
    if flag==0:
        unkown_index.append(index)
        unkown_gene_name.append(gene)

unkown_gene = pd.DataFrame(unkown_gene_name)
unkown_gene.to_csv(path_or_buf='../data/pan/string_850/novel_gene_2.0.txt', sep='\t', header=False,index=False)

In [4]:
#divide the train set and test set for predicting novel potential cancer driver genes
def create_k_sets():
    data = pd.read_table("../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt", index_col=0, header =0, sep="\t")
    length = len(data.values)
    index_array = np.arange(0,length,1)
    tr_list = [False]*length
    for suoyin1 in driver_index:
        tr_list[suoyin1] = True
    for suoyin2 in normal_index:
        tr_list[suoyin2] = True
        
    te_list = [False]*length
    for suoyin4 in unkown_index:
        te_list[suoyin4] = True
            
    inputs = []
    inputs.append(tr_list)
    inputs.append(te_list)
    k_sets = np.array(inputs)
    print(k_sets.shape)
    with open('../data/pan/string_850/k_sets_novel_2.0.pkl', 'wb') as f:
        pickle.dump(k_sets, f)
        
create_k_sets()